In [ ]:
import pandas as pd

In [ ]:
# 读取 questionnaire.csv, 存到 df 中，并将第一列（uid）设置为索引
question = pd.read_csv("questionnaire.csv",index_col='uid');
question.head(5);

In [ ]:
import numpy as np
#将所有uid随机分配到一个名为 user_mat 的10×10的numpy矩阵中
uids = question.index.values;
np.random.shuffle(uids)
user_mat = uids.reshape((10,10));

In [ ]:
#随机"shuffle" user_mat 十次，并将返回的“染色体”保存到 initial_population
initial_population = []

for x in range(10):
    user_mat = np.random.permutation(range(1, 101)).reshape((10,10))
    initial_population.append(user_mat)

In [ ]:
initial_population

In [ ]:
#写一个函数，输入是用户A（person）和一组用户B的数据（*others），返回A同B中每个用户喜好的平均曼哈顿距离
from scipy.spatial.distance import pdist
def calculate_distace(people):
    
def calculate_interest_distance(person, *others):
    people = [df.ix[person, 0:3]]
    for other in others:
        if other:
            people.append(df.ix[other, 0:3])

    return np.average(pdist(people, lambda x,o:np.sum(np.abs(people[0]-o)))) if len(people) > 1 else 0

In [ ]:
def get_element_safe(sequence, position):
    try:
        return sequence.item(position)
    except IndexError:
        return None

In [ ]:
#补全calculate_chromosome_fitness函数，该函数接收一个染色体，并为该染色体上的每个基因计算曼哈顿距离，
#最后以这些距离的平均值作为这条染色体的“适者程度”
def calculate_chromosome_fitness(chromosome):
    fitnesses = []
    for index, gene in np.ndenumerate(chromosome):
        next_to = calculate_interest_distance(gene,
                                             get_element_safe(chromosome, (index[0], index[1] - 1)),
                                             get_element_safe(chromosome, (index[0], index[1] + 1)))
        in_front = calculate_interest_distance(gene,
                                              get_element_safe(chromosome,(index[0]-1,index[1])))
        behind = calculate_interest_distance(gene,
                                            get_element_safe(choromosome,(index[0]+1,index[1])))
        fitnesses.append(sum([next_to, in_front, behind]) / 3)
    return np.average(fitnesses)

In [ ]:
generations = []

for chromosome in initial_population:
    generations.append({
        "generation": 0,
        "fitness": calculate_chromosome_fitness(chromosome),
        "chromosome": chromosome,
        "active": True
    })

In [ ]:
#以 fitness 为 key，为generations排序
generations = sorted(generations,key=lambda x:x['fitness'])
generations

In [ ]:
#补全”遗传规则“
import random

def recombine(mother, father):
    child = np.ones((10,10)) * -1
    # 遍历每个用户
    for index, uid in enumerate(df.index):
        if uid <= 50:
            # 如果uid <=50，则position为mother中该uid的位置
            position = np.where(mother == uid)
        else:
            # 如果uid > 50，则position为father中该uid的位置
            position = np.where(father == uid)
        
        child[position] = 0;
        if child[position] != -1:
            # 处理冲突
            filtered = np.where(np.equal(child, -1))
            position = random.choice(list(zip(filtered[0], filtered[1])))

        child[position] = uid

    return child

In [ ]:
import random

def mutate_chromosome(chromosome):
    first = (random.randrange(0, 10, 1), random.randrange(0, 10, 1))
    second = (random.randrange(0, 10, 1), random.randrange(0, 10, 1))

    chromosome[first[0], first[1]], chromosome[second[0], second[1]] = chromosome[second[0], second[1]], chromosome[first[0], first[1]]
    return chromosome

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def draw_population(population, generation):
    get_ids = np.vectorize(get_id)
    figure, axes = plt.subplots(1, 10, figsize=(20, 3))

    for index, chromosome in enumerate(population):
        ids = chromosome
        axes[index].imshow(ids, interpolation="nearest", cmap=plt.cm.plasma)
        subplot_title = "Chromosome " + str(index)
        axes[index].set_title(subplot_title)

    plot_title = "Generation " + str(generation)
    figure.suptitle(plot_title)
    plt.show()

In [ ]:
draw_population(initial_population, 0)

In [ ]:
import copy
#又到了交配的季节
"""
代数达到上限
后代的适应性超过阈值
后代适应性退化
"""
#代数上限
max_itea = N;
most_fit = []

# 当前的适应度
max_fitness = None

# 适应度最高的染色体
fittest_chromosome = None

fittest_chromosome = generations[0]['chromosome'];
max_fitness = generations[0]['fitness'];
most_fit.append(max_fitness);

i = 1;
while(True):
    if(generations[-1]['generation'] < 50):
        break;
    else:
        next_generations = [];
        np.random.shuffle(generations);
        for i in range(1,len(generations)):
            child = recombine(generations[i-1]['chromosome'],generations[i]['chromosome']);
            child =  mutate_chromosome(child);
            next_generations.append({
                "generation": i+1,
                "fitness": calculate_chromosome_fitness(child),
                "chromosome": child,
                "active": True
            })
        i  = i + 1;
        next_generations = sorted(next_generations,key=lambda x:x['fitness']);
        fittest_chromosome = next_generations[0]['chromosome'] if next_generations[0]['fitness'] < max_fitness else fittest_chromosome;
        max_fitness = min(max_fitness,next_generations[0]['fitness']);
        most_fit.append(next_generations[0]['fitness']);
        
        if(most_fit[-1] < most_fit[-2] or max_fitness < 5):
            break;

In [ ]:
### 从generations取出某一代的所有染色体，并排序
def get_chromosomes_in_generation(generations, generation_num):
    # Get only the chromosomes in this generation
    generation = list(filter(lambda k: k['generation'] == generation_num, 
                             generations))
    return sorted(generation, key=lambda k: k['fitness'])


### 更新 most_fit / max_fitness / fittest_chromosome 
def sort_chromosomes(generation, most_fit):
    max_fitness = generation[0]['fitness']
    most_fit.append(max_fitness)
    fittest_chromosome = generation[0]['chromosome']
    return most_fit, fittest_chromosome

In [ ]:
# 请将以下Comment 添加到代码的合适位置

def copy_chromosomes(generation, generations):
    new_generation = copy.deepcopy(generation)
    # C. 将适应度最差的两个染色体杀掉
    new_generation[-1]['active'] = False
    new_generation[-2]['active'] = False
    
    # D. 找到仍存活的染色体，并更新代数
    active = list(filter(lambda k: k['active'] == True, new_generation))
    for chromosome in active:
        chromosome['generation'] += 1
    
    
    generations.extend(active)
    return generations


def mate_chromosomes(generation, generations, generation_num):
    # A. 将适应度最高的两条染色体交配
    child = recombine(generation[0]['chromosome'], generation[1]['chromosome'])
    child = mutate_chromosome(child)
    generations.append({
      "active": True,
      "chromosome": child,
      "generation": generation_num + 1,
      "fitness": calculate_chromosome_fitness(child)
    })
    
    # E. 将适应度第三和第四高的两条染色体交配
    other = recombine(generation[2]['chromosome'], generation[3]['chromosome'])
    other = mutate_chromosome(chromosome)
    # B. 将新一代染色体加入到generations中并返回
    generations.append({
      "active": True,
      "chromosome": other,
      "generation": generation_num + 1,
      "fitness": calculate_chromosome_fitness(other)
    })
    return generations

In [ ]:
def run_ga(input_generations):
    fittest = []
    max_fitness = None
    fittest_chromo = None
    for generation_num in range(50):
        if max_fitness and max_fitness < 5:
            break

        generation = get_chromosomes_in_generation(input_generations, generation_num)
        fittest, fittest_chromo = sort_chromosomes(generation, most_fit)

        input_generations = copy_chromosomes(generation, input_generations)
        input_generations = mate_chromosomes(generation, input_generations, generation_num)

        draw_population(map(lambda d: d['chromosome'], 
                        get_chromosomes_in_generation(input_generations, generation_num)), generation_num)

    return fittest, fittest_chromo

In [ ]:
most_fit, fittest_chromosome = run_ga(generations)

In [ ]:
plt.plot(most_fit)
plt.title("Most Fit Trend")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.show()

In [ ]:
fittest_chromosome